## 1. Load Data

In [1]:
!pip install nltk

In [2]:
import nltk

In [3]:
# nltk.download()

In [4]:
from nltk.corpus import reuters
reuters_corpus = reuters.raw()
reuters_corpus = reuters_corpus.lower()
reuters_corpus = reuters_corpus.split("\n      ")
reuters_corpus = [sent.replace("\n", "").split(" ") for sent in reuters_corpus]

In [5]:
reuters_corpus = reuters_corpus[:1000]

In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [7]:
np.__version__, torch.__version__

('1.26.0', '2.1.0')

In [8]:
import matplotlib
matplotlib.__version__

'3.8.1'

In [9]:
#2. numeralization
#find unique words
flatten = lambda l: [item for sublist in l for item in sublist]
#assign unique integer
vocabs = list(set(flatten(reuters_corpus))) #all the words we have in the system - <UNK>
vocabs.append('<UNK>')

In [10]:
#create handy mapping between integer and word
word2index = {v:idx for idx, v in enumerate(vocabs)}
word2index['fear']

5927

In [11]:
index2word = {v:k for k, v in word2index.items()}

## 2. Prepare train data

In [12]:
#create pairs of center word, and outside word

def random_batch(batch_size, corpus):

    skipgrams = []

    #loop each corpus
    for doc in corpus:
        #look from the 2nd word until second last word
        for i in range(2, len(doc)-2):
            #center word
            center = word2index[doc[i]]
            #outside words = 2 words
            outside = (word2index[doc[i-2]], word2index[doc[i-1]], word2index[doc[i+1]], word2index[doc[i+2]])
            #for each of these two outside words, we gonna append to a list
            for each_out in outside:
                skipgrams.append([center, each_out])
                #center, outside1;   center, outside2
                
    random_index = np.random.choice(range(len(skipgrams)), batch_size, replace=False)
    
    inputs, labels = [], []
    for index in random_index:
        inputs.append([skipgrams[index][0]])
        labels.append([skipgrams[index][1]])
        
    return np.array(inputs), np.array(labels)
            
x, y = random_batch(2, reuters_corpus)

In [13]:
x.shape

(2, 1)

In [14]:
x

array([[4463],
       [2165]])

In [15]:
y.shape

(2, 1)

## 3.Negative Sampling

In [16]:
z = 0.001

In [17]:
#count
from collections import Counter

word_count = Counter(flatten(reuters_corpus))
word_count

#get the total number of words
num_total_words = sum([c for w, c in word_count.items()])
num_total_words

29743

In [18]:
unigram_table = []

for v in vocabs:
    uw = word_count[v] / num_total_words
    uw_alpha = int((uw ** 0.75) / z)
    unigram_table.extend([v] * uw_alpha)
    

## 4.Model

In [19]:
def prepare_sequence(seq, word2index):
    idxs = list(map(lambda w: word2index[w] if word2index.get(w) is not None else word2index['<UNK>'], seq))
    return torch.LongTensor(idxs)

In [20]:
import random

def negative_sampling(targets, unigram_table, k):
    batch_size = targets.shape[0]
    neg_samples = []
    for i in range(batch_size):  #(1, k)
        target_index = targets[i].item()
        nsample      = []
        while (len(nsample) < k):
            neg = random.choice(unigram_table)
            if word2index[neg] == target_index:
                continue
            nsample.append(neg)
        neg_samples.append(prepare_sequence(nsample, word2index).reshape(1, -1))
        
    return torch.cat(neg_samples) #batch_size, k

In [21]:
batch_size = 2
x, y = random_batch(batch_size, reuters_corpus)
x_tensor = torch.LongTensor(x)
y_tensor = torch.LongTensor(y)

In [22]:
k = 5
neg_samples = negative_sampling(y_tensor, unigram_table, k)

In [23]:
y_tensor[1]

tensor([604])

In [24]:
neg_samples[1]

tensor([5165, 3830,    0, 2682,    0])

In [25]:
class SkipgramNeg(nn.Module):
    
    def __init__(self, voc_size, emb_size):
        super(SkipgramNeg, self).__init__()
        self.embedding_center  = nn.Embedding(voc_size, emb_size)
        self.embedding_outside = nn.Embedding(voc_size, emb_size)
        self.logsigmoid        = nn.LogSigmoid()
    
    def forward(self, center, outside, negative):
        #center, outside:  (bs, 1)
        #negative       :  (bs, k)
        
        center_embed   = self.embedding_center(center) #(bs, 1, emb_size)
        outside_embed  = self.embedding_outside(outside) #(bs, 1, emb_size)
        negative_embed = self.embedding_outside(negative) #(bs, k, emb_size)
        
        uovc           = outside_embed.bmm(center_embed.transpose(1, 2)).squeeze(2) #(bs, 1)
        ukvc           = -negative_embed.bmm(center_embed.transpose(1, 2)).squeeze(2) #(bs, k)
        ukvc_sum       = torch.sum(ukvc, 1).reshape(-1, 1) #(bs, 1)
        
        loss           = self.logsigmoid(uovc) + self.logsigmoid(ukvc_sum)
        
        return -torch.mean(loss)

In [26]:
#test your model
emb_size = 2
voc_size = len(vocabs)
model = SkipgramNeg(voc_size, emb_size)

In [27]:
loss = model(x_tensor, y_tensor, neg_samples)

In [28]:
loss

tensor(0.6015, grad_fn=<NegBackward0>)

## 5.Training

In [29]:
import time

In [30]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [31]:
num_epochs = 10

start = time.time()

for epoch in range(num_epochs):
    
    #get batch
    input_batch, label_batch = random_batch(batch_size, reuters_corpus)
    input_tensor = torch.LongTensor(input_batch)
    label_tensor = torch.LongTensor(label_batch)
    
    #predict
    neg_samples = negative_sampling(label_tensor, unigram_table, k)
    loss = model(input_tensor, label_tensor, neg_samples)
    
    #backprogate
    optimizer.zero_grad()
    loss.backward()
    
    #update alpha
    optimizer.step()
    
    #print the loss
    #if (epoch + 1) % 1000 == 0:
    print(f"Epoch {epoch+1:6.0f} | Loss: {loss:2.6f}")

print(f"Time to train: {time.time() - start}")

Epoch      1 | Loss: 1.509367
Epoch      2 | Loss: 1.727324
Epoch      3 | Loss: 1.607196
Epoch      4 | Loss: 0.856919
Epoch      5 | Loss: 1.427237
Epoch      6 | Loss: 1.354551
Epoch      7 | Loss: 1.354863
Epoch      8 | Loss: 1.010693
Epoch      9 | Loss: 1.456220
Epoch     10 | Loss: 1.497438
Time to train: 3.106306791305542


## 6. Semantic and Syntatic

In [32]:
import pandas as pd

pd.read_csv("word-test.v1.txt", skiprows=0)

# analogies_path = datapath('word-test.v1.txt')
# word_analogies_data = open(analogies_path, 'r').readlines()

,// Copyright 2013 Google Inc. All Rights Reserved.
0,: capital-common-countries
1,Athens Greece Baghdad Iraq
2,Athens Greece Bangkok Thailand
3,Athens Greece Beijing China
4,Athens Greece Berlin Germany
...,...
19553,write writes talk talks
19554,write writes think thinks
19555,write writes vanish vanishes
19556,write writes walk walks


In [33]:
with open("word-test.v1.txt") as f:
    data = f.read()

In [34]:
data = data.replace("\t", "")
data = data.lower()
data = data.split(": ")

In [35]:
capital = data[1]
capital = capital.split('\n')[1:-1]
capital_row = [line.split(" ") for line in capital]

In [36]:
past = data[12]
past = past.split('\n')[1:-1]
past_row = [line.split(" ") for line in past]

In [37]:
def get_embed(word):
    try:
        index = word2index[word]
    except:
        word = '<UNK>'
        
    word = torch.LongTensor([word2index[word]])
    
    embed_c = model.embedding_center(word)
    embed_o = model.embedding_outside(word)
    embed   = (embed_c + embed_o) / 2
    embed = embed[0][0].item(), embed[0][1].item()
    
    return np.array(embed)

In [38]:
#more formally is to divide by its norm
def cosine_similarity(A, B):
    dot_product = np.dot(A, B)
    norm_a = np.linalg.norm(A)
    norm_b = np.linalg.norm(B)
    similarity = dot_product / (norm_a * norm_b)
    return similarity

In [39]:
negative_embeddings = {}

for vocab in vocabs:
    negative_embeddings[vocab] = get_embed(vocab)


In [47]:
import pickle

with open("embeddings_negative.pkl", 'wb') as f:
    pickle.dump(negative_embeddings, f)

In [41]:
past_correct = 0
for row in past_row:
   row = [word.lower() for word in row]
   w1, w2, w3, w4 = row
   
   embedding = get_embed(w3) - get_embed(w1) + get_embed(w2)
   similarities = {}

   for vocab in negative_embeddings.keys():
       similarities[vocab] = cosine_similarity(embedding, negative_embeddings[vocab])
       
   predicted_word = max(similarities, key=similarities.get)
   
   if predicted_word == w4:
       past_correct += 1
   
past_accuracy = past_correct / len(past_row)
print(past_accuracy)

0.0


In [42]:
capital_correct = 0
for row in capital_row:
   row = [word.lower() for word in row]
   w1, w2, w3, w4 = row
   
   embedding = get_embed(w3) - get_embed(w1) + get_embed(w2)
   similarities = {}

   for vocab in negative_embeddings.keys():
       similarities[vocab] = cosine_similarity(embedding, negative_embeddings[vocab])
       
   predicted_word = max(similarities, key=similarities.get)
   
   if predicted_word == w4:
       capital_correct += 1
   
capital_accuracy = capital_correct / len(capital_row)
print(capital_accuracy)

0.001976284584980237


In [43]:
import numpy as np
import pandas as pd

df = pd.read_csv("wordsim_similarity_goldstandard.txt", sep='\t', header=None)
df

,0,1,2
0,tiger,cat,7.35
1,tiger,tiger,10.00
2,plane,car,5.77
3,train,car,6.31
4,television,radio,6.77
...,...,...,...
198,rooster,voyage,0.62
199,noon,string,0.54
200,chord,smile,0.54
201,professor,cucumber,0.31


In [44]:
def get_word_vector(word, embs):
    try:
        return embs[word]
    except:
        return embs['<UNK>']

In [45]:
df['dot'] = df.apply(lambda row: np.dot(get_word_vector(row[0].lower(), negative_embeddings), get_word_vector(row[0].lower(), negative_embeddings)), axis=1)
df

,0,1,2,dot
0,tiger,cat,7.35,1.143019
1,tiger,tiger,10.00,1.143019
2,plane,car,5.77,1.143019
3,train,car,6.31,1.143019
4,television,radio,6.77,2.517996
...,...,...,...,...
198,rooster,voyage,0.62,1.143019
199,noon,string,0.54,1.143019
200,chord,smile,0.54,1.143019
201,professor,cucumber,0.31,1.143019


In [46]:
df.corr(numeric_only=True, method='spearman')

,2,dot
2,1.000000,0.068668
dot,0.068668,1.000000


## Model comparation between Skipgram, Skipgram (Neg), Glove and Glove (Gensim) model
| Model          | Window Size | Training Loss | Training Time | Syntactic Accuracy | Semantic Accuracy |
|----------------|-------------|---------------|---------------|--------------------|-------------------|
| Skipgram       |      2      |    9.948281   |     3.15 s    |         0 %        |        0 %        |
| Skipgram (NEG) |      2      |    1.497438   |     3.1 s     |        0.2 %       |        0 %        |
| Glove          |      2      |   80.799538   |      1 s      |         0 %        |        0 %        |
| Glove (Gensim) |      -      |       -       |       -       |       93.87 %      |      55.49 %      |
##  
##  
|   Model  | Skipgram |  NEG | Glove | Glove (gensim) |
|:--------:|----------|:----:|:-----:|----------------|
| SpearMan |   0.17   | 0.07 | -0.16 |      0.53      |